# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [1]:
import openai
import os

print ("OpenAI version:", openai.__version__)
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

OpenAI version: 1.30.5


In [2]:
# client = openai.OpenAI(api_key=OPENAI_KEY)
client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

def get_completion(prompt, model="llama3"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Here's a product description based on the technical specifications:

**Mid-Century Inspired Office Chair**

Elevate your workspace with our stylish and functional office chair, inspired by the classic designs of the mid-century era. This beautiful piece of furniture is part of a comprehensive family of office solutions, including filing cabinets, desks, bookcases, meeting tables, and more.

Choose from a range of shell colors and base finishes to match your unique style: stainless steel, matte black, gloss white, or chrome. The chair itself is available with or without armrests, ensuring comfort and support for any user.

The chair's construction features a sturdy 5-wheel plastic coated aluminum base, providing stability and ease of movement. The pneumatic chair adjust allows for effortless height adjustment, making it perfect for home or business settings.

**Specifications:**

* Dimensions: 53 cm wide x 51 cm deep x 80 cm high (20.87" x 20.08" x 31.50")
* Seat Height: 44 cm (17.32")


## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Do not output anything else. Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Elevate your workspace with our stylish mid-century inspired office chair. Choose from a range of shell colors and base finishes, including stainless steel, matte black, gloss white, or chrome. Select from 10 fabric and 6 leather upholstery options, or opt for a plastic back and front. With pneumatic adjustability and comfortable HD36 foam seating, this chair is perfect for home or business settings.


In [14]:
len(response.split(' '))

63

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [15]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Do not output anything else. Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair boasts a sturdy shell constructed from cast aluminum with a modified nylon PA6/PA66 coating, ensuring durability and resistance to scratches. The seat features high-density HD36 foam for optimal comfort. Choose from various base finishes, including stainless steel, matte black, gloss white, or chrome, and select from 10 fabric and 6 leather upholstery options.


In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Do not output anything else. Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair is crafted from high-quality materials, featuring a cast aluminum shell base with a modified nylon PA6/PA66 coating and a sturdy HD36 foam seat. Choose from various options, including armrests, soft or hard-floor casters, and two seat foam densities.

Product IDs: 5-wheel plastic coated aluminum base (SWC-100), pneumatic chair adjust (SWC-110)


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [22]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Here is the product description and dimensions table in HTML format:

<div>
This mid-century inspired office chair is part of a beautiful family of furniture designed to bring style and functionality to your home or business setting. Constructed from high-quality materials, this chair features a 5-wheel plastic coated aluminum base for stability and durability. The pneumatic chair adjust allows for easy raise and lower action, making it perfect for users of all heights.

The shell is made from cast aluminum with modified nylon PA6/PA66 coating, providing a sturdy and scratch-resistant surface. The seat is crafted from HD36 foam, ensuring comfort and support for long hours of use.

Choose from a range of options to customize your chair to suit your needs, including soft or hard-floor caster options, medium or high-density seat foam, and armless or 8-position PU armrests.

Product IDs: A7A3C5E, C2B1D4F, E5H9K2L

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Name</th>
   

## Load Python libraries to view HTML

In [10]:
from IPython.display import display, HTML

In [23]:
display(HTML(response))

Name,Measurement (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


## Try experimenting on your own!